In [1]:
import torch

In [2]:
N = 2
C = 3
H = 5
W = 5
x = torch.arange(N*H*W).reshape(N, H, W)*2
x

tensor([[[ 0,  2,  4,  6,  8],
         [10, 12, 14, 16, 18],
         [20, 22, 24, 26, 28],
         [30, 32, 34, 36, 38],
         [40, 42, 44, 46, 48]],

        [[50, 52, 54, 56, 58],
         [60, 62, 64, 66, 68],
         [70, 72, 74, 76, 78],
         [80, 82, 84, 86, 88],
         [90, 92, 94, 96, 98]]])

In [3]:
indices = torch.randint(H * W, (N, 8), dtype=torch.long)
indices, indices.shape

(tensor([[20,  6, 11, 14, 11, 12, 15, 19],
         [ 4,  6,  8, 14, 15,  3, 16, 18]]), torch.Size([2, 8]))

In [4]:
ga = torch.gather(x.view(N, H * W), 1, indices).view(N, -1)
ga, ga.shape

(tensor([[40, 12, 22, 28, 22, 24, 30, 38],
         [58, 62, 66, 78, 80, 56, 82, 86]]), torch.Size([2, 8]))

In [5]:
ga = ga / 100
ga, x 

(tensor([[0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0]]), tensor([[[ 0,  2,  4,  6,  8],
          [10, 12, 14, 16, 18],
          [20, 22, 24, 26, 28],
          [30, 32, 34, 36, 38],
          [40, 42, 44, 46, 48]],
 
         [[50, 52, 54, 56, 58],
          [60, 62, 64, 66, 68],
          [70, 72, 74, 76, 78],
          [80, 82, 84, 86, 88],
          [90, 92, 94, 96, 98]]]))

In [6]:
x2 = torch.arange(N*C*H*W).reshape(N, C, H, W)*2
x2[0][0], x2[0][1]

(tensor([[ 0,  2,  4,  6,  8],
         [10, 12, 14, 16, 18],
         [20, 22, 24, 26, 28],
         [30, 32, 34, 36, 38],
         [40, 42, 44, 46, 48]]), tensor([[50, 52, 54, 56, 58],
         [60, 62, 64, 66, 68],
         [70, 72, 74, 76, 78],
         [80, 82, 84, 86, 88],
         [90, 92, 94, 96, 98]]))

In [7]:
indices.unsqueeze(1).expand(-1, C, -1).shape

torch.Size([2, 3, 8])

In [8]:
ga = torch.gather(x2.view(N, C, -1), 2, indices.unsqueeze(1).expand(-1, C, -1))
ga, ga.shape

(tensor([[[ 40,  12,  22,  28,  22,  24,  30,  38],
          [ 90,  62,  72,  78,  72,  74,  80,  88],
          [140, 112, 122, 128, 122, 124, 130, 138]],
 
         [[158, 162, 166, 178, 180, 156, 182, 186],
          [208, 212, 216, 228, 230, 206, 232, 236],
          [258, 262, 266, 278, 280, 256, 282, 286]]]), torch.Size([2, 3, 8]))

In [9]:
ga[:, 0, :], ga[:, 1, :]

(tensor([[ 40,  12,  22,  28,  22,  24,  30,  38],
         [158, 162, 166, 178, 180, 156, 182, 186]]),
 tensor([[ 90,  62,  72,  78,  72,  74,  80,  88],
         [208, 212, 216, 228, 230, 206, 232, 236]]))

# Gather Backward

In [10]:
x3 = x2.float().requires_grad_(True)
ga = torch.gather(x3.view(N, C, -1), 2, indices.unsqueeze(1).expand(-1, C, -1))
x3.sum().backward()

In [11]:
ga

tensor([[[ 40.,  12.,  22.,  28.,  22.,  24.,  30.,  38.],
         [ 90.,  62.,  72.,  78.,  72.,  74.,  80.,  88.],
         [140., 112., 122., 128., 122., 124., 130., 138.]],

        [[158., 162., 166., 178., 180., 156., 182., 186.],
         [208., 212., 216., 228., 230., 206., 232., 236.],
         [258., 262., 266., 278., 280., 256., 282., 286.]]],
       grad_fn=<GatherBackward>)

In [12]:
N = 2
C = 3
H = 5
W = 5
x = torch.arange(N*H*W).reshape(N, H, W)*2
indices = torch.randint(H * W, (N, 30), dtype=torch.long)
ga = torch.gather(x.view(N, H * W), 1, indices).view(N, -1)
indices, ga

(tensor([[16,  5,  6,  8, 13,  3, 22, 16, 24, 16, 20, 18, 22, 11,  2,  3,  2, 12,
           4,  8, 23, 15, 12,  3,  2, 13, 10,  7,  0, 11],
         [ 5, 13,  7, 20,  6,  1, 19, 18, 17,  8, 11,  3, 10,  7,  9, 19, 19, 11,
          24,  7, 21, 10,  7,  1, 12, 21,  0, 17,  8, 12]]),
 tensor([[32, 10, 12, 16, 26,  6, 44, 32, 48, 32, 40, 36, 44, 22,  4,  6,  4, 24,
           8, 16, 46, 30, 24,  6,  4, 26, 20, 14,  0, 22],
         [60, 76, 64, 90, 62, 52, 88, 86, 84, 66, 72, 56, 70, 64, 68, 88, 88, 72,
          98, 64, 92, 70, 64, 52, 74, 92, 50, 84, 66, 74]]))

In [13]:
v, i = ga.topk(5)
print("Value ;", v, v.shape)
print("Idx ;",i, i.shape)

Value ; tensor([[48, 46, 44, 44, 40],
        [98, 92, 92, 90, 88]]) torch.Size([2, 5])
Idx ; tensor([[ 8, 20,  6, 12, 10],
        [18, 25, 20,  3, 15]]) torch.Size([2, 5])


In [14]:
reverse_indices = torch.gather(indices, 1, i)
reverse_indices

tensor([[24, 23, 22, 22, 20],
        [24, 21, 21, 20, 19]])

In [15]:
torch.gather(x.view(N, H * W), 1, reverse_indices).view(N, -1)

tensor([[48, 46, 44, 44, 40],
        [98, 92, 92, 90, 88]])

In [16]:
N = 2
C = 1
H = 4
W = 4
x = torch.arange(N*H*W).reshape(N, H, W)
indices = torch.randint(H * W, (N, 5), dtype=torch.long)

s = [2, 2]
x2 = torch.arange(N*H*W*s[0]*s[1]).reshape(N, H*s[0], W*s[1])
x, x2, indices

(tensor([[[ 0,  1,  2,  3],
          [ 4,  5,  6,  7],
          [ 8,  9, 10, 11],
          [12, 13, 14, 15]],
 
         [[16, 17, 18, 19],
          [20, 21, 22, 23],
          [24, 25, 26, 27],
          [28, 29, 30, 31]]]),
 tensor([[[  0,   1,   2,   3,   4,   5,   6,   7],
          [  8,   9,  10,  11,  12,  13,  14,  15],
          [ 16,  17,  18,  19,  20,  21,  22,  23],
          [ 24,  25,  26,  27,  28,  29,  30,  31],
          [ 32,  33,  34,  35,  36,  37,  38,  39],
          [ 40,  41,  42,  43,  44,  45,  46,  47],
          [ 48,  49,  50,  51,  52,  53,  54,  55],
          [ 56,  57,  58,  59,  60,  61,  62,  63]],
 
         [[ 64,  65,  66,  67,  68,  69,  70,  71],
          [ 72,  73,  74,  75,  76,  77,  78,  79],
          [ 80,  81,  82,  83,  84,  85,  86,  87],
          [ 88,  89,  90,  91,  92,  93,  94,  95],
          [ 96,  97,  98,  99, 100, 101, 102, 103],
          [104, 105, 106, 107, 108, 109, 110, 111],
          [112, 113, 114, 115, 116, 117

In [17]:
ys = (indices // W) * s[0]
xs = (indices % W) * s[1]

W *= s[1]

new_ind = (ys * W) + xs
print(indices, "\n", new_ind)

tensor([[10,  6,  5,  8,  1],
        [10,  2,  9,  9, 10]]) 
 tensor([[36, 20, 18, 32,  2],
        [36,  4, 34, 34, 36]])
